## reconstituting 4D matrices from individual 3D matrices

In [1]:
import napari
#from aicsimageio import AICSImage
from skimage import io
from medpy.io import load #to read the ACME outputs we need to use medpy. aicsimageio doesn't seem to be able to handle it

In [2]:
from os import listdir,walk

In [3]:
import numpy as np
from math import ceil, floor

In [4]:
viewer = napari.Viewer(title='s3_acme_labelled_smad6')

C:\Users\hkqur\miniconda3\envs\devbio_2\lib\site-packages\networkx\utils\backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [5]:
dir_pth = 'C:/Users/hkqur/Documents/napari/smad6_3/base_im/'

In [20]:
# reconstitute 4D image from disapparate ACME outputs so can all be labelling in one go in Napari.

class frame_rebuild(): #rebuilt functions in class form, so can instantiate multiple objects #2025-02-20 HQ - specify the directory path and the file extension you are searching for in the directory, in this case MHA
    # 2024-05-16: need to obtain biggest array and use that as template array that other smaller images need to pad up to. Just using the first image may not always work. #2025-02-19: HQ NB, indeed the approach wasn't succesful when frame sizes peaked beyond the firt frame so this has been reworked.
    
    def max_array_size(self, dir_path):
        directory_files = listdir(dir_pth)
        #directory_files
        #input file extension input argument here
        directory_images = [i for i in directory_files if i[len(i)-4:]=='.mha']
        
        array_size_list = []
        for a in range(0,(len(directory_images))):
            print(dir_pth,'{}'.format(directory_images[a]))
            loaded_image, loaded_metadata = load(dir_pth+str(directory_images[a]))
            loaded_image_shape = loaded_image.shape
            array_size_list.append(loaded_image_shape)
        #print(array_size_list)
        #max_array_size = max(array_size_list)
        
        # 2025-02-18 HQ - generate maximum possible frame size by taking max x y z from list of all dimensions
        max_size=(max([x[0] for x in array_size_list]),
        max([x[1] for x in array_size_list]),
        max([x[2] for x in array_size_list]))
        
        #2025-02-18: HQ reworked to work on max possible dimensions. #2025-02-24: HQ removed separate function and integrated with max_array_size function to work more smoothly in class (only require one input now)
        array_3d_list = []
        for a in range(0,(len(directory_images))):
            print(dir_pth,'{}'.format(directory_images[a]))
            loaded_image, loaded_metadata = load(dir_pth+str(directory_images[a]))
            #print(loaded_image)
            array_x,array_y,array_z=loaded_image.shape
            first_image_size_x,first_image_size_y,first_image_size_z = max_size
                
        
            if max_size == loaded_image.shape: #check if dimensions of previous image and current image are the same. if the same no processing needed, just add image to array and proceed
                #print('loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
                #image_array = np.stack((image_array,loaded_image), axis=-1)
                
                array_3d_list.append(loaded_image)
        
        
            else: #if dimensions are mismatched, centre the images
                #print('loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
                print('size mismatch')
                x_diff = abs(array_x-first_image_size_x)
                y_diff = abs(array_y-first_image_size_y)
                z_diff = abs(array_z-first_image_size_z)
                    
                print('first_image_size_z = {} z_diff= {}'.format(first_image_size_z,z_diff))
                y_padding = ceil(y_diff/2)
                x_padding = ceil(x_diff/2)
                z_padding = ceil(z_diff/2)
                print(array_z,first_image_size_z)
                print('z_padding= ', z_padding)
                    
                if z_diff%2!=0:
                    z_pad_left = ceil(z_diff/2)
                    z_pad_right = floor(z_diff/2)
                    #print("z_pad_left, ",z_pad_left, "z_pad_right, ", z_pad_right, "full, ", first_image_size[2], "mid, ", (first_image_size[2])-z_pad_right-z_pad_left)
                    #len(first_image_size[2])-z_pad_right
                    #print("first_image", (first_image_size[2]))
                    print("loaded_image", (loaded_image.shape[2]))
                    loaded_image = np.pad(loaded_image,((x_padding,x_padding),(y_padding,y_padding),(z_pad_left,z_pad_right)),'constant')
        
        
                else:       
                    loaded_image = np.pad(loaded_image,((x_padding,x_padding),(y_padding,y_padding),(z_padding,z_padding)),'constant')
        
                #np.pad(loaded_image,((x_diff,x_diff),(y_diff,y_diff),(z_diff,z_diff)),'constant')
                #print('following PAD- loaded_image: {}, image_array: {}'.format(loaded_image.shape, image_array.shape))
                #print("z_pad_left, ",z_pad_left, "z_pad_right, ", z_pad_right, "full, ", first_image_size[2], "mid, ", (first_image_size[2])-z_pad_right-z_pad_left)
                    
                #image_array = np.stack((image_array,loaded_image), axis=-1)
                #print("size check, ", loaded_image.shape, first_image_size)
                print("z_diff ",z_diff)
                array_3d_list.append(loaded_image)
        return np.array(array_3d_list)

        def __init__(self):
            return max_array_size(dir_path)
            

In [21]:
outarray = frame_rebuild()

In [24]:
im_array = outarray.max_array_size(dir_pth)

C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_001.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_002.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_003.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_004.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_005.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_006.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_007.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_008.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_009.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_010.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_011.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_012.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_013.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_014.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/base_im/ s3_015.czi.mha
C:/Users/hkqur/Documents/napari/smad6_3/

In [25]:
viewer.add_image(im_array,name='image_array')

<Image layer 'image_array' at 0x21089229fd0>

# to do

import the raw data to be overlaid by the imported ACME segmentation. Make sure that timepoints match up.